In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

plt.style.use('ggplot')
%matplotlib inline

x_train = pd.read_csv('data/x_train_norm.csv', sep=';', na_values='?')
x_test = pd.read_csv('data/x_test_norm.csv', sep=';', na_values='?')
y_train = pd.read_csv('data/y_train.csv', sep=';', header=None, na_values='?')[0]



import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

def scorer_logloss(estimator, X, y):
    return logloss(y, estimator.predict_proba(X)[:,1])

def revert_scorer_logloss(estimator, X, y):
    return -1 * scorer_logloss(estimator, X, y)

In [32]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV

In [5]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=11)
cross_val_score(classifier, x_train, y_train, n_jobs=3, scoring=revert_scorer_logloss)

array([-0.48055015, -0.45686146, -0.47958079])

In [6]:
good_idx = margin>-0.95 #1 #0.84
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1200, random_state=11, n_jobs=3)
classifier.fit(x_train[good_idx], y_train[good_idx])

scorer_logloss(classifier, x_train, y_train)
#crosses = cross_val_score(classifier, x_train[good_idx], y_train[good_idx], n_jobs=3, scoring=revert_scorer_logloss)
#print np.mean(crosses), crosses

NameError: name 'margin' is not defined

In [19]:
limit1 = 18000
limit2 = 20000
xx_train, yy_train = x_train[:limit1], y_train[:limit1]
xx_valid, yy_valid = x_train[limit1:limit2], y_train[limit1:limit2]
xx_test, yy_test = x_train[limit2:], y_train[limit2:]

In [21]:
clf = RandomForestClassifier(n_estimators=25)
clf.fit(xx_train, yy_train)
clf_probs = clf.predict_proba(xx_test)
score = log_loss(y_test, clf_probs)

In [29]:
#print log_loss(yy_test, clf_probs)
#print logloss(yy_test, clf_probs[:,1])

0.70311082909
0.703110980271


In [46]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=3)
clf.fit(xx_train, yy_train)
clf_probs = clf.predict_proba(xx_test)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(xx_valid, yy_valid)
sig_clf_probs = sig_clf.predict_proba(xx_test)
sig_score = log_loss(yy_test, sig_clf_probs)

In [47]:
print sig_score

0.395894551821


In [48]:
y_test = sig_clf.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/005_rf_1000_Calibrated.csv', sep=';', header=None, index=False)
#mlbootcamp=0,4016667

In [52]:
limit = 20000
xx_train, yy_train = x_train[:limit], y_train[:limit]
xx_valid, yy_valid = x_train[limit:], y_train[limit:]
clf = RandomForestClassifier(n_estimators=1000, n_jobs=3)
clf.fit(xx_train, yy_train)
#clf_probs = clf.predict_proba(xx_test)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(xx_valid, yy_valid)
#sig_clf_probs = sig_clf.predict_proba(xx_test)
#sig_score = log_loss(yy_test, sig_clf_probs)

y_test = sig_clf.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/005_rf_1000_Calibrated_all.csv', sep=';', header=None, index=False)
#mlbootcamp=0,4008066